In [4]:
from time import sleep

from IPython.display import clear_output

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import json

## Рейтинг лучших вузов России RAEX-100, 2022 год
Источник: https://raex-rr.com/pro/education/russian_universities/top-100_universities/2022/

Поля:
- Место
- Название
- Место, 2021 год
- Рейтинговый функционал (балл)
- Условия для получения качественного образования, ранг
- Уровень востребованности выпускников работодателями, ранг
- Уровень научно-исследовательской деятельности, ранг

In [2]:
url = 'https://raex-rr.com/pro/education/russian_universities/top-100_universities/2022/'

In [5]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
browser = webdriver.Chrome(chrome_options=options)

C:\Users\kamik\AppData\Local\Temp\ipykernel_1564\3051664040.py:3: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(chrome_options=options)


In [6]:
browser.get(url)

In [16]:
def concatUnName(parts):
    fullname = ''
    for part in parts: fullname += part + ' '
    return fullname[:-1] 

In [9]:
trlist = browser.find_elements(By.TAG_NAME, 'tr')

### Data stucture for Univercity
{key = midname (имя из таблицы на сайте):

               {inn: ИНН вуза

                region: регион расположения

                link: ссылка на сайт

                rank_2022: рейтинг в 2022 году

                rank_2021: рейтинг в 2021 году

                rank_full: pейтинговый функционал (балл)

                rank_edu: условия для получения качественного образования

                rank_job: уровень востребованности выпускников работодателями

                rank_sci: уровень научно-исследовательской деятельности
                
               }
}

In [32]:
all_un_info = {}
for i in range(1,len(trlist)):
    un_info = {}
    parts = trlist[i].text.split(' ')
    all_un_info[concatUnName(parts[1:-5])] = {'rank_2022':parts[0], 'rank_2021': parts[-5], 'rank_full': parts[-4], 'rank_edu': parts[-3], 'rank_job': parts[-2], 'rank_sci': parts[-1]}
    

In [33]:
all_un_info["Российский государственный университет нефти и газа (национальный исследовательский университет) имени И.М. Губкина"]

{'rank_2022': '26',
 'rank_2021': '26',
 'rank_full': '3.3341',
 'rank_edu': '31',
 'rank_job': '13',
 'rank_sci': '41'}

#### Extract univercity profile links

In [50]:
trlist = browser.find_elements(By.TAG_NAME, 'a')

In [68]:
prof_links = {}
for i in range(20,len(trlist)):
    prof_links[trlist[i].text] = trlist[i].get_attribute('href')

#### Extract information from profile

In [ ]:
for univer in all_un_info:
    print(univer)
    browser.get(prof_links[univer])
    trlist = browser.find_elements(By.CLASS_NAME,'database_company_main_info')
    inn = ''
    region = ''
    link = ''
    for i in range(0,len(trlist)):
        parts = trlist[i].text.split('\n')
        match parts[0]:
            case 'ИНН': inn = parts[1]
            case 'Регион': region = parts[1]
            case 'Сайт': link = parts[1]
    all_un_info[univer]['inn'] = inn
    # lambda-выражение проверяет, есть ли в начале названия региона "г. ". 
    # Если есть, то возвращает название региона без этой подстроки.
    # Само выражение срабатывает, если длина строки с регионом > 0.
    # Иными словами, если регион есть в профайле универиситета 
    all_un_info[univer]['region'] = ((lambda r: r[3:] if (r[0]+r[1]+r[2] == 'г. ') else r)(region)) if len(region)>0 else ''
    all_un_info[univer]['link'] = link
    sleep(10)
    

In [249]:
for univer in all_un_info:
    if (all_un_info[univer].get('region','none') == 'Хабаровск'): 
        print(univer)


Тихоокеанский государственный университет


In [251]:
all_un_info['Тихоокеанский государственный университет']['region'] = 'Хабаровский край'

#### Преобразование словаря в список для сохранения в файл

In [254]:
un_list = []
for univer in all_un_info:
    juniv = {}
    juniv['name'] = univer
    juniv['inn'] = all_un_info[univer]['inn']
    juniv['region'] = all_un_info[univer]['region']
    juniv['link'] = all_un_info[univer]['link']
    juniv['rank_2022'] = all_un_info[univer]['rank_2022']
    juniv['rank_2021'] = all_un_info[univer]['rank_2021']
    juniv['rank_full'] = all_un_info[univer]['rank_full']
    juniv['rank_edu'] = all_un_info[univer]['rank_edu']
    juniv['rank_job'] = all_un_info[univer]['rank_job']
    juniv['rank_sci'] = all_un_info[univer]['rank_sci']
    un_list.append(juniv)

In [255]:

import json

jf = json.dumps(un_list, indent = 3, ensure_ascii=False)

In [256]:
with open('univercity_rating.json', 'w') as file:
    json.dump(un_list,file, indent = 3, ensure_ascii = False)